In [ ]:
import tensorflow as tf

# **Importing Important Libraries**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib 
import os 
import PIL
from PIL import Image

# **Loading and Preprocessing Data**

In [2]:
data_dir = pathlib.Path("Custom-Dataset")
# print(len(list(data_dir.glob("*/*"))))


In [ ]:
trash = list(data_dir.glob('Can/*'))
PIL.Image.open(str(trash[0]))

In [4]:
batch_size = 32
image_height = 224
image_width = 224

In [40]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  # validation_split=0.1,
  # subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

Found 23343 files belonging to 3 classes.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

Found 21000 files belonging to 3 classes.
Using 2100 files for validation.


# **Configure Dataset Performance**

In [28]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds= test_ds.prefetch(buffer_size=AUTOTUNE)

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
tf.get_logger().setLevel('ERROR')

In [6]:
model = tf.keras.models.load_model("Models/ResNet50-STBv1.0_13")


In [20]:
model = tf.keras.models.load_model("Models/ResNet50-STBv1.0_13")

In [31]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                             

In [46]:
def build_model(hp):
    model = tf.keras.models.load_model("Models/ResNet50-STBv1.0_13")
    
    # Setting Resnet50 to be false
    for layer in model.layers[2].layers:
        layer.trainable = False

    if hp.Choice("conv1",[True,False]):
        for layer in model.layers[2].layers[1:7]:
            layer.trainable = True
        
    if hp.Choice("conv2_block1",[True,False]):
        for layer in model.layers[2].layers[7:19]:
            layer.trainable = True
    
    if hp.Choice("conv2_block2",[True,False]):
        for layer in model.layers[2].layers[19:29]:
            layer.trainable = True

    if hp.Choice("conv2_block3",[True,False]):
        for layer in model.layers[2].layers[29:39]:
            layer.trainable = True
    
    if hp.Choice("conv3_block1",[True,False]):
        for layer in model.layers[2].layers[39:51]:
            layer.trainable = True

    if hp.Choice("conv3_block2",[True,False]):
        for layer in model.layers[2].layers[51:61]:
            layer.trainable = True
    
    if hp.Choice("conv3_block3",[True,False]):
        for layer in model.layers[2].layers[61:71]:
            layer.trainable = True

    if hp.Choice("conv3_block4",[True,False]):
        for layer in model.layers[2].layers[71:81]:
            layer.trainable = True

    if hp.Choice("conv4_block1",[True,False]):
        for layer in model.layers[2].layers[81:93]:
            layer.trainable = True
    
    if hp.Choice("conv4_block2",[True,False]):
        for layer in model.layers[2].layers[93:103]:
            layer.trainable = True
    
    if hp.Choice("conv4_block3",[True,False]):
        for layer in model.layers[2].layers[103:113]:
            layer.trainable = True
    
    if hp.Choice("conv4_block4",[True,False]):
        for layer in model.layers[2].layers[113:123]:
            layer.trainable = True
    
    if hp.Choice("conv4_block5",[True,False]):
        for layer in model.layers[2].layers[123:133]:
            layer.trainable = True
    
    if hp.Choice("conv4_block6",[True,False]):
        for layer in model.layers[2].layers[133:143]:
            layer.trainable = True
    
    if hp.Choice("conv5_block1",[True,False]):
        for layer in model.layers[2].layers[143:155]:
            layer.trainable = True

    if hp.Choice("conv5_block2",[True,False]):
        for layer in model.layers[2].layers[155:165]:
            layer.trainable = True
    
    if hp.Choice("conv5_block3",[True,False]):
        for layer in model.layers[2].layers[165:]:
            layer.trainable = True
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    return model

In [9]:
from kerastuner.engine.hyperparameters import HyperParameters

# Define the hyperparameters
hp = HyperParameters()

<ipython-input-9-d3216093274a>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.engine.hyperparameters import HyperParameters


In [ ]:
%pip install keras-tuner --upgrade

In [11]:
from kerastuner import Hyperband

In [12]:
from keras_tuner import Objective

In [47]:
tuner = Hyperband(
    build_model,
    objective=Objective('val_accuracy', direction='max'),
    max_epochs=25,
    factor=3,
    overwrite=False,
    directory='Tuner/ResNet50-STBv1.0_15',
    project_name='HB_tuner')

In [48]:
tuner.search(train_ds, epochs=25, validation_data=test_ds)

Trial 25 Complete [15h 00m 24s]
val_accuracy: 0.625

Best val_accuracy So Far: 0.6458333134651184
Total elapsed time: 4d 04h 12m 07s

Search: Running Trial #26

Value             |Best Value So Far |Hyperparameter
1                 |1                 |conv1
0                 |0                 |conv2_block1
0                 |0                 |conv2_block2
1                 |1                 |conv2_block3
1                 |1                 |conv3_block1
1                 |1                 |conv3_block2
1                 |1                 |conv3_block3
1                 |1                 |conv3_block4
1                 |0                 |conv4_block1
1                 |0                 |conv4_block2
0                 |1                 |conv4_block3
0                 |0                 |conv4_block4
0                 |1                 |conv4_block5
0                 |0                 |conv4_block6
1                 |0                 |conv5_block1
1                 |1        

Epoch 10/25
730/730 [==============================] - 3481s 5s/step - loss: 0.0183 - accuracy: 0.9937 - val_loss: 2.5864 - val_accuracy: 0.6146
Epoch 11/25
730/730 [==============================] - 3487s 5s/step - loss: 0.0194 - accuracy: 0.9946 - val_loss: 3.4475 - val_accuracy: 0.5208
Epoch 12/25
404/730 [===============>..............] - ETA: 25:59 - loss: 0.0169 - accuracy: 0.9954

In [33]:
# Create a function to convert labels to one-hot encoding
num_classes = 3
def to_one_hot(x, y):
    num_classes = 3
    y_one_hot = tf.one_hot(y, num_classes)
    return x, y_one_hot

In [36]:
train_ds = train_ds.map(to_one_hot)

In [116]:
val_ds = val_ds.map(to_one_hot)

In [37]:
test_ds = test_ds.map(to_one_hot)

## Evaluation and Predictions

In [42]:
# Evaluating on a new dataset from the internet
# paper and plastic from  https://www.kaggle.com/datasets/arthurcen/waste-images-from-sushi-restaurant
# can from https://universe.roboflow.com/dataset-t7hz7/cans-fdboa/dataset/1
# test_dir = pathlib.Path("Dataset-testing (from internet)")
# test_dir = pathlib.Path("WasteImagesDataset")
test_dir = pathlib.Path("data-camera-rpi")


batch_size = 32
image_height = 224
image_width = 224

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)


Found 96 files belonging to 3 classes.


In [ ]:
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)

In [ ]:
num_models = 30
all_models = tuner.get_best_models(num_models=num_models)

In [ ]:
tuner_results = {}
for i,model in enumerate(all_models):
    print(f"{i}-best model")
    l,p = model.evaluate(test_ds)
    tuner_results[i] = [l,p]
    

In [ ]:
best_model_1 = all_models[14]
best_model_1.save("Models/ResNet50-STBv1.0_12")

In [125]:
import json
with open('tuner_result.json', 'w') as fp:
    json.dump(tuner_results, fp)

In [ ]:
num_trials = 15
from kerastuner import HyperParameters
tuner_hps_summary = tuner.get_best_hyperparameters(num_trials=num_trials)

In [ ]:
summary_dict = {}
for i,a in enumerate(tuner_hps_summary):
  print(f"Best trial {i}:")

  trial_dict = {}
  for hp_name, hp_value in a.values.items():
      # print(f"{hp_value}")
      trial_dict[hp_name] = hp_value
  summary_dict[i] = trial_dict

In [ ]:
import pandas as pd

# Create a pandas DataFrame from the dictionary
df = pd.DataFrame.from_dict(summary_dict, orient='index')

# Write the DataFrame to an excel file
df.to_excel('/content/drive/My Drive/Models/output-3_1.xlsx', na_rep='')

In [ ]:
print("Best hyperparameters:")

for i,trial in enumerate(tuner_hps_summary):
  print(f"Trial {i}")
  for hp_name, hp_value in trial.values.items():
      print(f"- {hp_name}: {hp_value}")

## Convert to TfLite

In [2]:
model = keras.models.load_model("Models/ResNet50-STBv1.0_13")

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('Models/ResNet50-STBv1.0_13.Lite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
model.summary()